# 1. Introduction

In this project, I will build a machine learning classifier that predicts condition of water wells in Tanzania using information about the well, such as the type of pump, when it was installed, etc.


# 2. Business Understanding

### 2 (a) Problem Statement

MSABI, one of the largest water, sanitation, and hygiene (WASH) non-profit organizations in Tanzania, has a mission to improve access to clean water for  all communities in the country. 

Despite their efforts, many water wells in Tanzania still face challenges such as being in need of repair or failing altogether, which limits access to clean water for the population.

And with a population of 57 million, it is crucial to ensure that the country's water wells are functional and provide safe drinking water to the people.

The problem that MSABI faces is that they must prioritise their efforts and allocate resources effectively to address this issue.

But, currently, MSABU has limited information about the condition of water wells in the country making it difficult to determine which wells are most in need of attention.

### 2 (b) Main Objective

To build a classifier that can predict the codition of water wells in Tanzania and improve access to clean water for communities in the country.

### 2 (c) Specific Objectives

* Explore and understand the data related to water wells in Tanzania
* Prepocess the data and prepare it for building the classifier
* Build a machine learning model that can classify the condition of waterpoint as functional, functional but in need of repair, or non-functional
* Evaluate performance of the model and fine-tune it as needed
* Make predictions on new, unseeen data and evaluate the performance of the model
* Use the best-performing model to inform the efforts of MSABI and as a result improve access to clean water in Tanzania.


# 3. Data Understanding

The project utilizes data from Taarifa and Tanzanian Ministry of Water. The data includes information on 59,400 waterpoints.

There are three datasets available for this project, the training set labels, the training set values, and the test set values.

The target variable is "status_group" which details the functionality of waterpoints is a ternary classification classified as functional, functional but in need of repair, or non-functional.

The dataset includes the following features:
* amount_tsh - Total static head (amount water available to waterpoint)
* date_recorded - The date the row was entered
* funder - Who funded the well
* gps_height - Altitude of the well
* installer - Organization that installed the well
* longitude - GPS coordinate
* latitude - GPS coordinate
* wpt_name - Name of the waterpoint if there is one
* num_private -
* basin - Geographic water basin
* subvillage - Geographic location
* region - Geographic location
* region_code - Geographic location (coded)
* district_code - Geographic location (coded)
* lga - Geographic location
* ward - Geographic location
* population - Population around the well
* public_meeting - True/False
* recorded_by - Group entering this row of data
* scheme_management - Who operates the waterpoint
* scheme_name - Who operates the waterpoint
* permit - If the waterpoint is permitted
* construction_year - Year the waterpoint was constructed
* extraction_type - The kind of extraction the waterpoint uses
* extraction_type_group - The kind of extraction the waterpoint uses
* extraction_type_class - The kind of extraction the waterpoint uses
* management - How the waterpoint is managed
* management_group - How the waterpoint is managed
* payment - What the water costs
* payment_type - What the water costs
* water_quality - The quality of the water
* quality_group - The quality of the water
* quantity - The quantity of water
* quantity_group - The quantity of water
* source - The source of the water
* source_type - The source of the water
* source_class - The source of the water
* waterpoint_type - The kind of waterpoint
* waterpoint_type_group - The kind of waterpoint

# 4. Data Preparation

### Import Libraries

In [176]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

### Load Data

In [177]:
train_values = pd.read_csv("Training set values.csv")
train_labels = pd.read_csv("Training set labels.csv")

In [178]:
test_df=pd.read_csv("Test set values.csv")
test_copy=test.copy()

### Examine the Data

In [179]:
# Preview Train Values
train_values.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [180]:
# Preview Train labels
train_labels.head(2)

,id,status_group
0,69572,functional
1,8776,functional


In [181]:
# merge train labels and train values
train_df = train=train_values.merge(train_labels,on='id',how='inner')

In [182]:
#Preview the merged data set
train_df.head(2)

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [183]:
#check number of rows and columns
train_df.shape

(59400, 41)

In [184]:
#test dimension
test_df.shape

(14850, 40)

In [185]:
#summary information of merged dataframe
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [186]:
#summary statistics of numerical columns of dataframe
train_df.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,59400.000000,59400.000000,59400.000000,59400.000000,5.940000e+04,59400.000000,59400.000000,59400.000000,59400.000000,59400.000000
mean,37115.131768,317.650385,668.297239,34.077427,-5.706033e+00,0.474141,15.297003,5.629747,179.909983,1300.652475
std,21453.128371,2997.574558,693.116350,6.567432,2.946019e+00,12.236230,17.587406,9.633649,471.482176,951.620547
min,0.000000,0.000000,-90.000000,0.000000,-1.164944e+01,0.000000,1.000000,0.000000,0.000000,0.000000
25%,18519.750000,0.000000,0.000000,33.090347,-8.540621e+00,0.000000,5.000000,2.000000,0.000000,0.000000
50%,37061.500000,0.000000,369.000000,34.908743,-5.021597e+00,0.000000,12.000000,3.000000,25.000000,1986.000000
75%,55656.500000,20.000000,1319.250000,37.178387,-3.326156e+00,0.000000,17.000000,5.000000,215.000000,2004.000000
max,74247.000000,350000.000000,2770.000000,40.345193,-2.000000e-08,1776.000000,99.000000,80.000000,30500.000000,2013.000000


### Data Preprocessing

Here I will perform data cleaning, transformation, and normalizations to make it ready for further processing and to ensure quality and accuracy of the results. 

Here are the main steps I will follow:
* Remove missing values
* Handling outliers
* Splitting data into training and testing sets
* Encoding target variable

#### i) Handle missing values

In the data summary above, I noticed there were missing values in the following columns:

* funder
* installer
* subvillage
* public_meeting
* scheme_management
* scheme_name
* permit

I will focus on each column, one by one and handle missing values appropriately.

The process will as well soem feature engineering where I will group the data into smaller categories that are more useful for our analysis.

##### i. a) Funder

In [187]:
#funder
funder = train_df["funder"].value_counts()
funder_count = len(funder)
print("There are", funder_count, "funders.\n")
print("***********************************\n")
print("Here are the top 10 funders\n")
print("***********************************\n")
print(funder.head(10))


There are 1897 funders.

***********************************

Here are the top 10 funders

***********************************

Government Of Tanzania    9084
Danida                    3114
Hesawa                    2202
Rwssp                     1374
World Bank                1349
Kkkt                      1287
World Vision              1246
Unicef                    1057
Tasaf                      877
District Council           843
Name: funder, dtype: int64


I will divide the funder column into 6 categories. Each of the 5 top funders will remain as a category on its own and all the others will be grouped under others.

In [188]:
#function to divide funder column into 6 categories
#Top 5 funders will remain as a category and the rest grouped as others
def funder_categorised(train_df):
    if train_df["funder"]=="Goverment of Tanzania":
        return "government"
    elif train_df["funder"] == "Danida":
        return "danida"
    elif train_df["funder"] == "Hesawa":
        return "hesawa"
    elif train_df["funder"] == "Rwssp":
        return "rwssp"
    elif train_df["funder"] == "World Bank":
        return "world_bank"
    else:
        return "other"
    
#replace data with these new categories in both test and train data
train_df["funder"] = train_df.apply(lambda row: funder_categorised(row), axis = 1)
test_df["funder"] = test_df.apply(lambda row: funder_categorised(row), axis = 1)        
    

In [189]:
#doublecheck new categories
train_df["funder"].value_counts()

other         51361
danida         3114
hesawa         2202
rwssp          1374
world_bank     1349
Name: funder, dtype: int64

##### i. b) installer 

In [190]:
#installer
installer = train_df["installer"].value_counts()
installer_count = len(installer)
print("There are", installer_count, "installers.\n")
print("***********************************\n")
print("Here are the top 10 installers\n")
print("***********************************\n")
print(installer.head(10))


There are 2145 installers.

***********************************

Here are the top 10 installers

***********************************

DWE                   17402
Government             1825
RWE                    1206
Commu                  1060
DANIDA                 1050
KKKT                    898
Hesawa                  840
0                       777
TCRS                    707
Central government      622
Name: installer, dtype: int64


Similarly, I will divide the installer column into 6 categories. Each of the 5 top installers will remain as a category on its own and all the others will be grouped under others.

In [191]:
#function to divide installer column into 6 categories
#Top 5 installers will remain as a category and the rest grouped as others
def installer_categorised(train_df):
    if train_df["installer"]=="DWE":
        return "dwe"
    elif train_df["installer"] == "Government":
        return "government"
    elif train_df["installer"] == "RWE":
        return "rwe"
    elif train_df["installer"] == "Commu":
        return "commu"
    elif train_df["installer"] == "DANIDA":
        return "danida"
    else:
        return "other"
    
#replace data with these ew categories in both test and train data
train_df["installer"] = train_df.apply(lambda row: installer_categorised(row), axis = 1)
test_df["installer"] = test_df.apply(lambda row: installer_categorised(row), axis = 1)        


In [192]:
#doublecheck new categories
train_df["installer"].value_counts()

other         36857
dwe           17402
government     1825
rwe            1206
commu          1060
danida         1050
Name: installer, dtype: int64

With these new categories, let's see if we still have null values on the columns installer and funder

In [193]:
#double check missing values for funder column
missing_values_funder = train_df["funder"].isnull().sum()
print("The funder column has", missing_values_funder, "missing values.\n")
missing_values_installer = train_df["installer"].isnull().sum()
print("The installer column has", missing_values_funder, "missing values.\n")

The funder column has 0 missing values.

The installer column has 0 missing values.



##### i. c) subvillage

In [194]:
#subvillage
subvillage = train_df["subvillage"].value_counts()
subvillage_count = len(subvillage)
print("There are", subvillage_count, "subvillages.\n")
print("***********************************\n")
print("Here are the top 10 subvillages\n")
print("***********************************\n")
print(subvillage.head(10))

There are 19287 subvillages.

***********************************

Here are the top 10 subvillages

***********************************

Madukani    508
Shuleni     506
Majengo     502
Kati        373
Mtakuja     262
Sokoni      232
M           187
Muungano    172
Mbuyuni     164
Mlimani     152
Name: subvillage, dtype: int64


Unlike  the installers and funders columns, there is not much difference between subvillages counts thus I cannot really categorise and rank the subvillages. 

I think it is best I drop this column

In [195]:
#dropping subvillage column
train_df=train_df.drop(['subvillage'],axis=1)
test_df=test_df.drop(['subvillage'],axis=1)

##### i. d) public_meeting

In [196]:
#public_meeting
public_meeting = train_df["public_meeting"].value_counts()
print(public_meeting)
missing_values_public_meeting = train_df["public_meeting"].isnull().sum()
print("\nThe public_meeting column has", missing_values_public_meeting, "missing values.\n")

True     51011
False     5055
Name: public_meeting, dtype: int64

The public_meeting column has 3334 missing values.



I will replace the missing values with 'unknown' for both test and train.



In [199]:
train_df.public_meeting = train.public_meeting.fillna('Unknown')
test_df.public_meeting = test.public_meeting.fillna('Unknown')

In [200]:
train_df["public_meeting"].value_counts()

True       51011
False       5055
Unknown     3334
Name: public_meeting, dtype: int64

In [201]:
#double check missing values for public_meeting column
missing_values_public_meeting = train_df["public_meeting"].isnull().sum()
print("\nThe public_meeting column has", missing_values_public_meeting, "missing values.\n")



The public_meeting column has 0 missing values.



##### i. a) Encoding Target Variable

In [155]:
# convert target variable into numerical data 
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#le.fit(target_variable)
#target_variable_encoded = le.transform(target_variable)